<a href="https://colab.research.google.com/github/adityabhalerao22/system_threat_forecaster/blob/main/system_threat_forecaster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

system_threat_forecaster_path = kagglehub.competition_download('System-Threat-Forecaster')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("/kaggle/input/System-Threat-Forecaster/train.csv")

In [ ]:
#dropping target column
X = train.drop(columns=['target', 'MachineID'])
y = train['target']

In [ ]:
train.head()

In [ ]:
train.describe()

In [ ]:
train.shape

In [ ]:
train.dtypes

In [ ]:
# Check class distribution
plt.figure(figsize=(6, 4))
train['target'].value_counts().plot(kind='bar', color=['blue', 'red'])
plt.xlabel("Target Classes (0 = No Malware, 1 = Malware)")
plt.ylabel("Count")
plt.title("Class Distribution of Target Variable")
plt.show()


In [ ]:
# The bar chart above shows a balanced data set with almost an equal number of outcomes of Target.

In [ ]:
 #Identifying the columns segregating categorical and numerical data
catcolumns = X.select_dtypes(include=['object']).columns
numcolumns = X.select_dtypes(include=['int64', 'float64']).columns


In [ ]:
catcolumns

In [ ]:
numcolumns

In [ ]:
# Find columns with only one unique value
constant_columns = [col for col in X.columns if X[col].nunique() == 1]
constant_columns

In [ ]:
# These columns have 1 variable so we can remove them
train.drop(columns=['IsBetaUser'])
train.drop(columns=['IsFlightsDisabled'])
train.drop(columns=['AutoSampleSubmissionEnabled'])

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 12))
sns.heatmap(train.isnull(), cbar=False, cmap='viridis')
plt.title("Missing Values Heatmap")
plt.show()

In [ ]:
missing_values = train.isnull().sum()
print(missing_values)

In [ ]:
# Ignoring columns with 0 null values

In [ ]:
missing_values = train.isnull().sum()
missing_values = missing_values[missing_values > 0].sort_values(ascending=False)
print(missing_values)

In [ ]:
train[numcolumns] = train[numcolumns].replace([np.inf, -np.inf], np.nan)  # Replace inf values
train[numcolumns] = train[numcolumns].fillna(0)

In [ ]:
plt.figure(figsize=(15, 10))
for i, column in enumerate(numcolumns[:5], 1):  # Plot first 6 numerical features
    plt.subplot(2, 3, i)
    sns.histplot(train[column], bins=50, kde=True)
    plt.title(f"Distribution of {column}")
plt.tight_layout()
plt.show()

IsBetaUSER has only 1 variable

Most users have only 1-2 anti virus products installed

RealTimeProtectionState - distribution is highly skewed towards values 6-8

In [ ]:
nunique_values = train["SMode"].nunique()
nunique_values


In [ ]:
unique_values = train["SMode"].unique()
unique_values

In [ ]:
value_counts = train["SMode"].value_counts()
print(value_counts)

In [ ]:
correlation = train[["SMode", "target"]].corr()
print(correlation)


In [ ]:
s_mode_effect = train.groupby("SMode")["target"].mean()
print(s_mode_effect)


In [ ]:
s_mode_effect = train.groupby("IsSystemProtected")["target"].mean()
print(s_mode_effect)

In [ ]:

s_mode_effect = train.groupby("OSVersion")["target"].mean()
print(s_mode_effect)

In [ ]:
value_counts = train["OSVersion"].value_counts()
print(value_counts)

In [ ]:
# This indicates 10.0.4.0 has 100% malware rate
# This indicates 10.0.1.0, 10.0.2.0, 10.0.3.0 has 0% malware rate
#This is because of low sample size

In [ ]:
value_counts = train["PlatformType"].value_counts()
print(value_counts)

In [ ]:
s_mode_effect = train.groupby("PlatformType")["target"].mean()
print(s_mode_effect)

In [ ]:
# Windows2016 has the lowest probability for malware but it is still has a very low sample size

In [ ]:
value_counts = train["IsVirtualDevice"].value_counts()
print(value_counts)

In [ ]:
s_mode_effect = train.groupby("IsVirtualDevice")["target"].mean()
print(s_mode_effect)

In [ ]:
value_counts = train["OSArchitecture"].value_counts()
print(value_counts)

In [ ]:
s_mode_effect = train.groupby("OSArchitecture")["target"].mean()
print(s_mode_effect)

In [ ]:
s_mode_effect = train.groupby("PowerPlatformRole")["target"].mean()
print(s_mode_effect)

In [ ]:
value_counts = train["PowerPlatformRole"].value_counts()
print(value_counts)

In [ ]:
# FInding variance of each column

In [ ]:
variance_values = X[numcolumns].var()
print(variance_values)

In [ ]:
#Using KNN Imputer for numerical columns and using simple imputer for categorical columns
from sklearn.impute import KNNImputer
knnimputer = KNNImputer(n_neighbors=7)
X[numcolumns] = knnimputer.fit_transform(X[numcolumns])


In [ ]:
from sklearn.impute import SimpleImputer
catimputer = SimpleImputer(strategy='most_frequent')

In [ ]:
X[catcolumns] = catimputer.fit_transform(X[catcolumns])


In [ ]:
#Now converting the categorical values to numerical values (trying out various methods)
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder, MinMaxScaler
label_encoders = {}
for col in catcolumns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    label_encoders[col] = le

In [ ]:
# Scaling the numerical columns
scaler = StandardScaler()
X[numcolumns] = scaler.fit_transform(X[numcolumns])

In [ ]:
# using pipelining instead

In [ ]:
#from sklearn.pipeline import Pipeline
#from sklearn.compose import ColumnTransformer

#num_pipeline = Pipeline([('imputer', KNNImputer(n_neighbors=7)),('scaler', StandardScaler())])

#cat_pipeline = Pipeline([('imputer', SimpleImputer(strategy='most_frequent')),('encoder', OneHotEncoder(handle_unknown='ignore'))

# Combining both pipelines using ColumnTransformer
#preprocessor = ColumnTransformer([('num', num_pipeline, numcolumns),('cat', cat_pipeline, catcolumns)])

# Fit and transform the dataset
#X_transformed = preprocessor.fit_transform(X)

# Convert back to a DataFrame
#X= pd.DataFrame(X_transformed)


In [ ]:
train_log = train[numcolumns].apply(lambda x: np.log1p(x))
plt.figure(figsize=(15, 8))
sns.boxplot(data=train_log, orient='h')
plt.title("Box and Whisker Plot for Numerical Features")
plt.xlabel("Values")
plt.ylabel("Features")
plt.show()


In [ ]:
# Calculate the outliers

In [ ]:

def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)  # First quartile
    Q3 = df[column].quantile(0.75)  # Third quartile
    IQR = Q3 - Q1  # Interquartile Range
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Identifying outliers
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]

    print(f"Column: {column}")
    print(f"Q1: {Q1}, Q3: {Q3}")
    print(f"IQR: {IQR}")
    print(f"Lower Bound: {lower_bound}, Upper Bound: {upper_bound}")
    print(f"Number of outliers: {len(outliers)}")

    return outliers


In [ ]:
outliers = detect_outliers_iqr(train, "AntivirusConfigID")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

X['DateOS'] = pd.to_datetime(X['DateOS']).map(lambda x: x.toordinal()) #was getting an error so converted to numeric

corr_matrix = X.corr().abs()
sns.heatmap(corr_matrix)
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
 # Replacing NaN with the most frequent value
y.fillna(y.mode()[0], inplace=True)

In [ ]:
from sklearn.feature_selection import VarianceThreshold

var_thresh = VarianceThreshold(threshold=0.01)
X_train = var_thresh.fit_transform(X_train)
X_test = var_thresh.transform(X_test)

In [ ]:
# Model 1: Decision Tree with Hyperparameter Tuning

from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(random_state=42)
param_grid_dt = {'max_depth': [20, 30],'min_samples_split': [2, 5],'min_samples_leaf': [1, 2]}
dtc

In [ ]:
# Tuning using GridSearchCV

In [ ]:
grid_search_cv = GridSearchCV(dtc, param_grid_dt, cv=3, scoring='accuracy', n_jobs=-1)
grid_search_cv.fit(X_train, y_train)

best_params_cv = grid_search_cv.best_params_
best_cv = grid_search_cv.best_estimator_
cv_accuracy = round(best_cv.score(X_test, y_test), 2)

In [ ]:
# GridSearchCV is testing all possible combinations instead of doing it manually

In [ ]:
cv_accuracy

In [ ]:
best_cv

In [ ]:
best_params_cv

In [ ]:
# model 2 with Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=100, max_depth=30, random_state=42)
rfc.fit(X_train, y_train)
rfc_accuracy = rfc.score(X_test, y_test)
rfc_accuracy

In [ ]:
# model 3 with XGBoost

In [ ]:
import xgboost as xgb

xgb_model = xgb.XGBClassifier(n_estimators=300, max_depth=10, learning_rate=0.05, random_state=42)
xgb_model.fit(X_train, y_train)
xgb_accuracy = xgb_model.score(X_test, y_test)
xgb_accuracy

In [ ]:
# model 4 using lgb

In [ ]:
import lightgbm as lgb

lgb_model = lgb.LGBMClassifier(n_estimators=500, max_depth=15, learning_rate=0.05, random_state=42)
lgb_model.fit(X_train, y_train)

lgb_accuracy = lgb_model.score(X_test, y_test)
print("LightGBM Accuracy:", lgb_accuracy)


In [ ]:
# out of the all models tested, lgb model provides the highest score out of all.

In [ ]:
# Loading the test dataset and dropping MachineID
test = pd.read_csv("/kaggle/input/System-Threat-Forecaster/test.csv")
test_data = test.drop(columns=['MachineID'])

In [ ]:
test_data[catcolumns] = catimputer.transform(test_data[catcolumns])

In [ ]:
knn_imputer = KNNImputer(n_neighbors=5)
test_data[numcolumns] = knn_imputer.fit_transform(test_data[numcolumns])


In [ ]:
#applying the same label encoding for test
for col in catcolumns:
    if col in label_encoders:
        test_data[col] = test_data[col].astype(str).map(
            lambda s: label_encoders[col].transform([s])[0] if s in label_encoders[col].classes_ else -1
        )



In [ ]:
test_data[numcolumns] = scaler.transform(test_data[numcolumns])
test_data = var_thresh.transform(test_data)

In [ ]:
lgb_predictions = lgb_model.predict(test_data)

In [ ]:
# Plot confusion matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

y_pred = lgb_model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["No Malware", "Malware"], yticklabels=["No Malware", "Malware"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix - LightGBM")
plt.show()

Inference from confusion matrix

1. Accuracy
The model has an accuracy of 63.13%.

2. Precision
62.7% of predicted malware cases are actually malware.

3. Recall
66.8% of actual malware cases are detected correctly.

4. F1 score is 64.7%

In [ ]:
import lightgbm as lgb

lgb_model = lgb.LGBMClassifier(n_estimators=1000, max_depth=15, learning_rate=0.01, random_state=42)
lgb_model.fit(X_train, y_train)

lgb_accuracy = lgb_model.score(X_test, y_test)
print("LightGBM Accuracy:", lgb_accuracy)


In [ ]:
#saving the csv file in submission.csv

In [ ]:
submission = pd.DataFrame({'target': lgb_predictions})
submission.index.name = 'id'
submission.to_csv('/kaggle/working/submission.csv')